In [1]:
from langchain_tools import get_tools
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

MEMORY_KEY = "chat_history"

llm = ChatOpenAI(model="gpt-3.5-turbo")
tools = get_tools()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            As a professional nutritionist, please first answer the question using your existing knowledge. 
            Keep your answer precise, brief, and professional in tone.
            """,
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        (
            "user", 
            """
            Answer the following question or finish the task: {input}. 
            If you need to use a function to assist with your response, please search in 'English' and 
            inform me about your sources of information and the logic behind your recommendations.
            """
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

def execute_chat(input_message, chat_history):
    result = agent_executor.invoke({"input": input_message, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=input_message),
            AIMessage(content=result["output"]),
        ]
    )
    return result

In [2]:
result = agent_executor.invoke({"input": "我想吃牛肉", "chat_history": []})



> Entering new AgentExecutor chain...

Invoking: `find_restaurant_food` with `{'query': 'beef'}`


[{'food_name': 'Beef Stick, Grass Fed', 'brand_name_item_name': 'Chomps Beef Stick, Grass Fed', 'brand_name': 'Chomps'}, {'food_name': 'Mini Beef Stick, Original', 'brand_name_item_name': 'Country Archer Mini Beef Stick, Original', 'brand_name': 'Country Archer'}, {'food_name': 'No Seeds Soft Rye Bread', 'brand_name_item_name': 'Beefsteak No Seeds Soft Rye Bread', 'brand_name': 'Beefsteak'}, {'food_name': 'Beef Teriyaki', 'brand_name_item_name': 'Healthy Choice Beef Teriyaki', 'brand_name': 'Healthy Choice'}, {'food_name': 'Beef Merlot', 'brand_name_item_name': 'Healthy Choice Beef Merlot', 'brand_name': 'Healthy Choice'}, {'food_name': 'Ground Beef', 'brand_name_item_name': "Laura's Lean Beef Ground Beef", 'brand_name': "Laura's Lean Beef"}, {'food_name': 'Beef Steak, Original', 'brand_name_item_name': "Jack Link's Beef Steak, Original", 'brand_name': "Jack Link's"}, {'food_name': 'Bee

In [4]:
result

{'input': '我想吃牛肉',
 'chat_history': [],
 'output': "I found several beef options available for you to enjoy:\n1. Chomps Beef Stick, Grass Fed by Chomps\n2. Country Archer Mini Beef Stick, Original by Country Archer\n3. Beefsteak No Seeds Soft Rye Bread by Beefsteak\n4. Healthy Choice Beef Teriyaki by Healthy Choice\n5. Laura's Lean Beef Ground Beef by Laura's Lean Beef\n\nYou can explore these options to satisfy your craving for beef."}

In [191]:
chat_history = []

# 第一次对话
result1 = execute_chat("我想吃牛肉", chat_history)



> Entering new AgentExecutor chain...

Invoking: `find_restaurant_food` with `{'query': 'beef'}`


[{'food_name': 'Beef Stick, Grass Fed', 'brand_name_item_name': 'Chomps Beef Stick, Grass Fed', 'brand_name': 'Chomps'}, {'food_name': 'Mini Beef Stick, Original', 'brand_name_item_name': 'Country Archer Mini Beef Stick, Original', 'brand_name': 'Country Archer'}, {'food_name': 'No Seeds Soft Rye Bread', 'brand_name_item_name': 'Beefsteak No Seeds Soft Rye Bread', 'brand_name': 'Beefsteak'}, {'food_name': 'Beef Teriyaki', 'brand_name_item_name': 'Healthy Choice Beef Teriyaki', 'brand_name': 'Healthy Choice'}, {'food_name': 'Beef Merlot', 'brand_name_item_name': 'Healthy Choice Beef Merlot', 'brand_name': 'Healthy Choice'}, {'food_name': 'Ground Beef', 'brand_name_item_name': "Laura's Lean Beef Ground Beef", 'brand_name': "Laura's Lean Beef"}, {'food_name': 'Beef Steak, Original', 'brand_name_item_name': "Jack Link's Beef Steak, Original", 'brand_name': "Jack Link's"}, {'food_name': 'Bee

In [192]:
# 第二次对话
result2 = execute_chat("營養價值", chat_history)



> Entering new AgentExecutor chain...

Invoking: `get_nutrition_with_nlp` with `{'query': 'beef'}`


[{'food_name': 'beef', 'brand_name': None, 'serving_qty': 3, 'serving_unit': 'oz', 'serving_weight_grams': 85, 'nf_calories': 220.15, 'nf_total_fat': 14.1, 'nf_saturated_fat': 5.76, 'nf_cholesterol': 73.95, 'nf_sodium': 42.5, 'nf_total_carbohydrate': 0, 'nf_dietary_fiber': 0, 'nf_sugars': 0, 'nf_protein': 22.19, 'nf_potassium': 230.35, 'nf_p': 153, 'metadata': {'is_raw_food': False}, 'tags': {'item': 'beef', 'measure': None, 'quantity': '3.0', 'food_group': 2, 'tag_id': 192}, 'alt_measures': [{'serving_weight': 85, 'measure': 'oz', 'seq': 1, 'qty': 3}, {'serving_weight': 285, 'measure': 'piece, cooked, excluding refuse (yield from 1 lb raw meat with refuse)', 'seq': 2, 'qty': 1}, {'serving_weight': 100, 'measure': 'g', 'seq': None, 'qty': 100}]}]A 3 oz (85g) serving of beef provides approximately the following nutritional values:

- **Calories:** 220.15
- **Total Fat:** 14.1g
  - Satu